In [1]:

import datetime
import json
import time
from typing import Union

import requests
from dotenv import dotenv_values
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.tools import tool
from langchain_groq import ChatGroq

API_KEYS = dotenv_values(".env")

SECTORS_API_KEY = API_KEYS['SECTORS_API_KEY']
GROQ_API_KEY = API_KEYS['GROQ_API_KEY']

In [90]:
def sectors_api(url: str, params: dict = {}, json_output: bool = True) -> Union[str | dict]:
    """Get data from API, returns json formatted data or an python dictionary.
    """
    headers = {"Authorization": SECTORS_API_KEY}

    try:
        if params:
            response = requests.get(url, headers=headers, params=params)
        else:
            response = requests.get(url, headers=headers)
        data = response.json()
    except requests.exceptions.HTTPError as err:
        raise SystemExit(err)
    if json_output:
        return json.dumps(data).replace('\\', '')
    else:
        return data

@tool 
def get_helper_data(
    helper: str
    ) -> str:
    """Get list of subsectors, industries, or subindustries.Choose "subsectors", "industries", or "subindustries" for helper parameter. 
    """
    data = sectors_api(f'https://api.sectors.app/v1/{helper}/')
    return data

@tool 
def get_companies_by_index(
    index: str
    ) -> str:
    """Get list of companies by stock index.
    """
    data = sectors_api(f'https://api.sectors.app/v1/index/{index}/')
    return data

@tool 
def get_companies_by_subsector(
    subsector: str
    ) -> str:
    """Get list of companies by subsector.
    """
    data = sectors_api(f'https://api.sectors.app/v1/companies/', {"sub_sector":subsector})
    return data

@tool 
def get_companies_by_subindustry(
    subindustry: str
    ) -> str:
    """Get list of companies by subindustry.
    """
    data = sectors_api(f'https://api.sectors.app/v1/companies/', {"sub_industry":subindustry})
    return data

@tool 
def get_company_performance(
    stock: str
    ) -> str:
    """Get company's performance since IPO. Return percentage gain since listing date of a given stock ticker of the company.
    """
    data = sectors_api(f'https://api.sectors.app/v1/listing-performance/{stock}/')
    return data

@tool 
def get_company_report(
    stock: str,
    section: str = "all"
    ) -> str:
    """Get company's report. Filter the data using section parameter. List of sections: overview, valuation, future, peers, financials, dividend, management, ownership. Use comma to separate each section when retrieving data of more than one section, for example "valuation,dividend".
    """
    if section == "all":
        data = sectors_api(f'https://api.sectors.app/v1/company/report/{stock}/')
    else:
        data = sectors_api(f'https://api.sectors.app/v1/company/report/{stock}/', params={"sections":section})
    return data

@tool
def get_company_revenue_and_cost_segment(
    stock: str
) -> str:
    """Get revenue and cost segments of a company by given stock ticker."""
    data = sectors_api(f'https://api.sectors.app/v1/company/get-segments/{stock}/')
    return data

@tool
def get_subsector_report(
    sub_sector: str,
    section: str = None
) -> str:
    """Get report of a given subsector, organized into distinct sections. Filter the section using section parameter, leave empty if you want to get all sections."""
    if section == "all":
        data = sectors_api(f'https://api.sectors.app/v1/subsector/report/{sub_sector}/')
    else:
        data = sectors_api(f'https://api.sectors.app/v1/subsector/report/{sub_sector}/', params={"sections":section})
    return data

@tool
def get_top_companies_by_volume(
    start_date: str, 
    end_date: str, 
    top_n: int = 5
) -> str:
    """
    Get top companies by transaction volume. Filter the date using start_date and end_date with format "YYYY-MM-DD". Input top_n parameter to get n top companies.
    """
    data = sectors_api(f"https://api.sectors.app/v1/most-traded/?start={start_date}&end={end_date}&n_stock={top_n}")
    return data

@tool
def get_top_companies_ranked(
    year: int,
    classifications: str = "all",
    top_n: int = 5,
) -> str:
    """
    Get a list of top stock tickers in a given year based on certain classification (dividend_yield, total_dividend, revenue, or earnings).
    """
    data = sectors_api(f"https://api.sectors.app/v1/most-traded/?calssifications={classifications}&n_stock={top_n}&year={year}")
    return data

@tool
def get_daily_transaction(
    stock: str,
    start_date: str,
    end_date: str,
) -> str:
    """
    Get daily transaction data of a given stock ticker between start_date and end_date using "YYYY-DD-MM" format.
    """
    data = sectors_api(f"https://api.sectors.app/v1/daily/{stock}/?start={start_date}&end={end_date}")
    return data

@tool
def get_index_daily(
    index_code: str,
    start_date: str,
    end_date: str,
) -> str:
    """
    Get daily transaction data of a given index code between start_date and end_date. Date format is "YYYY-MM-DD".
    """
    data = sectors_api(f"https://api.sectors.app/v1/index-daily/{index_code}/?start={start_date}&end={end_date}")
    return data

@tool
def compare(
    num1: float,
    num2: float
) -> str:
    """Compare two numbers"""
    return f"{num1} is {'less' if num1<num2 else 'greater'} than {num2}"

@tool
def sum_numbers(*args: float) -> float:
    """Sum multiple numbers"""
    return sum(args)

@tool
def divide(
    dividend: float,
    divisor: float
    ) -> str:
    """Divide numbers"""
    return dividend/divisor

def date_to_text(date):
    return f"{date.strftime(r'%A')}, {date.strftime(r'%d').lstrip('0')} {date.strftime(r'%B %Y')}: {date.strftime(r'%Y-%m-%d')}"

tools = [
    compare,
    sum_numbers,
    divide,
    # get_helper_data, 
    # get_companies_by_index, 
    # get_companies_by_subsector, 
    # get_companies_by_subindustry, 
    get_company_performance, 
    get_company_report, 
    get_company_revenue_and_cost_segment, 
    get_subsector_report, 
    get_top_companies_by_volume, 
    get_top_companies_ranked, 
    get_daily_transaction, 
    get_index_daily,
]

In [93]:
llm = ChatGroq(
    temperature=0,
    model_name="llama3-groq-70b-8192-tool-use-preview",
    groq_api_key=GROQ_API_KEY,
)

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Answer the following queries, as if you are a financial robo-advisor, being as factual and analytical as you can. Use the right tools provided. Use the tools output for analysis, if the tools doesn't return anything, don't make things up. Refer the current time if the question relates to the current time.",
        ),
        ("human", "{input} Today is {current_time}"),
        MessagesPlaceholder("agent_scratchpad"),
    ]
)

# Exercise 1

In [94]:
EXCERCISE_1 = [
    "What are the top 5 companies by transaction volume on the first of this month?",
    "What are the most traded stock yesterday?",
    "What are the most traded stock on last friday?", # I added this query because the market is closed on weekends,
    "What are the top 7 most traded stocks between 6th June to 10th June this year?"
]

In [95]:
today = datetime.datetime.today()

In [96]:
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

for query in EXCERCISE_1:
    print("Question:", query)
    result = agent_executor.invoke(
        {
            "input": query,
            "current_time": date_to_text(today)
            }
        )
    print("Answer:", "\n", result["output"], "\n\n======\n\n")

Question: What are the top 5 companies by transaction volume on the first of this month?


> Entering new AgentExecutor chain...

Invoking: `get_top_companies_by_volume` with `{'start_date': '2024-08-01', 'end_date': '2024-08-01', 'top_n': 5}`


{"2024-08-01": [{"symbol": "GOTO.JK", "company_name": "PT GoTo Gojek Tokopedia Tbk", "volume": 1425057600, "price": 53}, {"symbol": "BIPI.JK", "company_name": "PT Astrindo Nusantara Infrastruktur Tbk.", "volume": 521955600, "price": 61}, {"symbol": "NICL.JK", "company_name": "PT PAM Mineral Tbk", "volume": 362601100, "price": 252}, {"symbol": "WIKA.JK", "company_name": "PT Wijaya Karya (Persero) Tbk", "volume": 314787400, "price": 220}, {"symbol": "DEWA.JK", "company_name": "Darma Henwa Tbk", "volume": 294681400, "price": 67}]}The top 5 companies by transaction volume on the first of this month are as follows:
1. PT GoTo Gojek Tokopedia Tbk with a volume of 1,425,057,600.
2. PT Astrindo Nusantara Infrastruktur Tbk. with a volume of 521,955,600.

# Exercise 2

In [97]:

query_1 = "What are the top 3 companies by transaction volume over the last 7 days?"
query_2 = "Based on the closing prices of BBCA between 1st and 30th of June 2024, are we seeing an uptrend or downtrend? Try to explain why."
query_3 = "What is the company with the largest market cap between BBCA and BREN? For said company, retrieve the email, phone number, listing date and website for further research."
query_4 = "What is the performance of GOTO (sym bol: GOTO) since its IPO listing?"
query_5 = "If i had invested into GOTO vs BREN on their respective IPO listing date, which one would have given me a better return over a 90 day horizon?"

EXCERCISE_2 = [query_1, query_2, query_3, query_4, query_5]

In [98]:
for query in EXCERCISE_2:
    print("Question:", query)
    result = agent_executor.invoke(
        {
            "input": query,
            "current_time": date_to_text(today)
            }
        )
    print("Answer:", "\n", result["output"], "\n\n======\n\n")

Question: What are the top 3 companies by transaction volume over the last 7 days?


> Entering new AgentExecutor chain...

Invoking: `get_top_companies_by_volume` with `{'start_date': '2024-07-28', 'end_date': '2024-08-04', 'top_n': 3}`


{"2024-07-29": [{"symbol": "GOTO.JK", "company_name": "PT GoTo Gojek Tokopedia Tbk", "volume": 1621366300, "price": 54}, {"symbol": "ATLA.JK", "company_name": "PT Atlantis Subsea Indonesia Tbk", "volume": 1255103900, "price": 50}, {"symbol": "BSBK.JK", "company_name": "PT Wulandari Bangun Laksana Tbk", "volume": 754254000, "price": 69}], "2024-07-30": [{"symbol": "GOTO.JK", "company_name": "PT GoTo Gojek Tokopedia Tbk", "volume": 1223172300, "price": 55}, {"symbol": "BSBK.JK", "company_name": "PT Wulandari Bangun Laksana Tbk", "volume": 792338900, "price": 69}, {"symbol": "BIPI.JK", "company_name": "PT Astrindo Nusantara Infrastruktur Tbk.", "volume": 629061500, "price": 59}], "2024-07-31": [{"symbol": "GOTO.JK", "company_name": "PT GoTo Gojek Tokope